In [9]:
from sys import path as sys_path
from os import chdir, curdir
from os import path as os_path

from glob import iglob
from itertools import product

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

HOME = os_path.expanduser("~")
chdir(f"{HOME}/src/topology-programming")

sys_path.insert(0, "src/")
sys_path.insert(0, "src/utilities")

# from onset.utilities import plotters as my_plotters

report_folder = "data/archive/crossfire-rerun-02-01-24/reports/"
ECMP_Pallet = [sns.color_palette("tab10")[1], sns.color_palette("tab10")[0]]
MCF_Pallet = sns.color_palette("tab10")[2:4]


def export_legend(legend, filename="legend"):
    fig = legend.figure
    fig.canvas.draw()
    bbox = legend.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(filename + ".pdf", dpi="figure", bbox_inches=bbox)


# Parameter dictionary
# plt.rcParams

result_df = pd.DataFrame()
all_columns = set()
for data_file in iglob(report_folder + "*.csv"):
    df = pd.read_csv(data_file)
    all_columns.update(df.columns)
    result_df = pd.concat([result_df, df], ignore_index=True)

result_df = result_df.reindex(columns=all_columns)
networks = {"bellCanada": "Bell Canada", "sprint": "Sprint", "surfNet": "Surf Net", "ANS": "ANS", "CRL": "CRL"}
result_df["Network"] = result_df["Experiment"].apply(lambda x: x.split("_")[0] if isinstance(x, str) else x)
result_df["Network"] = result_df["Network"].apply(lambda x: networks[x])
result_df["Strategy"] = result_df.apply(
    lambda row: row["Routing"] if row["Iteration"].split("-")[-1] == "0" else f"{row['Routing']}+ONSET", axis=1
)
result_df["Attack Scale"] = result_df["Experiment"].apply(lambda x: "100E9" if "100E9" in x else "200E9")

In [10]:
for network, scale, routing in product(
    result_df["Network"].unique(), result_df["Attack Scale"].unique(), result_df["Strategy"].unique()
):
    temp = result_df[result_df["Network"] == network]
    data = temp[temp["Attack Scale"] == scale]

    baseline_df = data[data["Strategy"] == routing]
    onset_df = data[data["Strategy"] == routing + "+ONSET"]

    if routing == "ECMP":
        pallet = ECMP_Pallet
    if routing == "MCF":
        pallet = MCF_Pallet

    # baseline_df = np.array(data_1["Congestion"])
    # onset_df = np.array(data_2["Congestion"])

    # data_dict = {"ECMP": X_1, "ECMP+ONSET": X_2}

    # my_plotters.congestion_multi_cdf(
    #     data_dict, inLabel="Max. Link Utilization", fig_name=f"data/plots/crossfire/congestion-{network}-{routing}-{scale}"
    # )

    plt.rc("font", size=22)
    figurename = f"data/plots/crossfire/congestion-{network}-{routing}-{scale}".replace(" ", "_")
    # )
    ax1 = sns.ecdfplot(data=baseline_df, x="Congestion", linestyle="--", linewidth=3, color=pallet[0])
    ax2 = sns.ecdfplot(data=onset_df, x="Congestion", linewidth=3, color=pallet[1])
    # Set up the initial legend.
    my_legend = plt.legend(
        [routing, routing + "+ONSET"],
        bbox_to_anchor=(-0.2, 2),
        loc="upper left",
        borderaxespad=0,
        ncol=2,
        frameon=False,
    )
    # handles, labels = my_legend.get_legend_handles_labels()
    plt.tick_params(length=8)
    export_legend(my_legend, figurename + "_legend")
    plt.legend().remove()
    plt.axvline(1, linestyle="--", color="black", linewidth=2)
    plt.ylabel("CDF")
    plt.xlabel("Max. Congestion")
    plt.tight_layout()
    plt.savefig(figurename + ".pdf")
    print(f"saved: {figurename}.pdf")

    with open(figurename + "_stats.txt", "w") as fob:
        fob.write("\tLoss Events\ttotal")
        fob.write(f"ONSET\t{len(onset_df[onset_df['Loss'] > 0])}\t{len(onset_df)}")
        fob.write(f"ONSET\t{len(onset_df[onset_df['Loss'] > 0])}\t{len(onset_df)}")
    print(f"saved: {figurename}_stats.txt")

In [ ]:
np.array(data_2["Congestion"])

array([0.793238, 0.474032, 0.170498, 0.488298, 0.652963, 0.572302,
       0.527005, 0.498216, 0.72343 , 0.757374, 0.710392, 0.516759,
       0.747662, 0.621343, 0.66092 , 0.789586, 0.6255  , 0.663951,
       0.730918, 0.634806, 0.475533, 0.722594, 0.780958])

In [ ]:
data_2[["Network", "Strategy", "Attack Scale", "Congestion", "Loss", "Throughput"]]

,Network,Strategy,Attack Scale,Congestion,Loss,Throughput
1,Bell Canada,ECMP+ONSET,100E9,0.793238,0.0,1.0
11,Bell Canada,ECMP+ONSET,100E9,0.474032,0.0,1.0
36,Bell Canada,ECMP+ONSET,100E9,0.170498,0.0,1.0
114,Bell Canada,ECMP+ONSET,100E9,0.488298,0.0,1.0
202,Bell Canada,ECMP+ONSET,100E9,0.652963,0.0,1.0
204,Bell Canada,ECMP+ONSET,100E9,0.572302,0.0,1.0
223,Bell Canada,ECMP+ONSET,100E9,0.527005,0.0,1.0
277,Bell Canada,ECMP+ONSET,100E9,0.498216,0.0,1.0
310,Bell Canada,ECMP+ONSET,100E9,0.723430,0.0,1.0
349,Bell Canada,ECMP+ONSET,100E9,0.757374,0.0,1.0
